# Book Recommendation System

## In this part we will work on Popularity Based Recommender System [The Easy One]. Meaning we'll write a code to find out the top books based on user rating and show them to the use. 

In [45]:
import pandas as pd
import numpy as np

**Dataset Loding**

In [46]:
prepro_data = pd.read_csv("/kaggle/input/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv")

In [47]:
prepro_data.head(3)

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada


In [48]:
prepro_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0           1031175 non-null  int64  
 1   user_id              1031175 non-null  int64  
 2   location             1031175 non-null  object 
 3   age                  1031175 non-null  float64
 4   isbn                 1031175 non-null  object 
 5   rating               1031175 non-null  int64  
 6   book_title           1031175 non-null  object 
 7   book_author          1031174 non-null  object 
 8   year_of_publication  1031175 non-null  float64
 9   publisher            1031175 non-null  object 
 10  img_s                1031175 non-null  object 
 11  img_m                1031175 non-null  object 
 12  img_l                1031175 non-null  object 
 13  Summary              1031175 non-null  object 
 14  Language             1031175 non-null  object 
 15

In [49]:
print(prepro_data.shape)
prepro_data.isnull().sum()

(1031175, 19)


Unnamed: 0                 0
user_id                    0
location                   0
age                        0
isbn                       0
rating                     0
book_title                 0
book_author                1
year_of_publication        0
publisher                  0
img_s                      0
img_m                      0
img_l                      0
Summary                    0
Language                   0
Category                   0
city                   14103
state                  22798
country                35374
dtype: int64

In [50]:
prepro_data.duplicated().sum()

0

# Let's do profiling 

In [51]:
from ydata_profiling import ProfileReport # this will help us to generate report on our dataset.

#now we have to create profile report

report = ProfileReport(prepro_data, title = 'Report of prepro_data',explorative =True)
report.to_file('/kaggle/working/Book_Report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Popularity based Recommender System

In [52]:
prepro_data.head(3)

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada


In [53]:
# Drop the unnecessary cols
new_book= prepro_data.drop(columns=['Unnamed: 0','location','age','year_of_publication','publisher','img_s','img_l','Summary','Language','city','state','country'])

In [54]:
new_book.head(3)

,user_id,isbn,rating,book_title,book_author,img_m,Category
0,2,0195153448,0,Classical Mythology,Mark P. O. Morford,http://images.amazon.com/images/P/0195153448.0...,['Social Science']
1,8,0002005018,5,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,['Actresses']
2,11400,0002005018,0,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,['Actresses']


In [55]:
# the plan is that we'll count tot num of rating that each book has
# we group the dataframe from the pov of book_title
new_book.groupby('book_title').count()


,user_id,isbn,rating,book_author,img_m,Category
book_title,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",4,4,4,4,4,4
Always Have Popsicles,1,1,1,1,1,1
Apple Magic (The Collector's series),1,1,1,1,1,1
"Ask Lily (Young Women of Faith: Lily Series, Book 5)",1,1,1,1,1,1
Beyond IBM: Leadership Marketing and Finance for the 1990s,1,1,1,1,1,1
...,...,...,...,...,...,...
Ã?Â?lpiraten.,2,2,2,2,2,2
Ã?Â?rger mit Produkt X. Roman.,4,4,4,4,4,4
Ã?Â?sterlich leben.,1,1,1,1,1,1


In [56]:
# Now we are interested in taking the rating col values's count
# there is a string in the rating col so we converted it into numeric

rating_count = new_book.groupby('book_title')['rating'].count().reset_index()
rating_count.rename(columns={'rating':'rating_count'},inplace = True) #because remember it's not rating but counts of rating 

In [57]:
rating_count

,book_title,rating_count
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241085,Ã?Â?lpiraten.,2
241086,Ã?Â?rger mit Produkt X. Roman.,4
241087,Ã?Â?sterlich leben.,1
241088,Ã?Â?stlich der Berge.,3


In [58]:
# Now let us find the avg value
avg = new_book.groupby('book_title')['rating'].mean().reset_index()
avg.rename(columns={'rating':'avg_rating'},inplace = True)

In [59]:
avg

,book_title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241085,Ã?Â?lpiraten.,0.000000
241086,Ã?Â?rger mit Produkt X. Roman.,5.250000
241087,Ã?Â?sterlich leben.,7.000000
241088,Ã?Â?stlich der Berge.,2.666667


In [60]:
popu_book = rating_count.merge(avg,on='book_title')

In [61]:
popu_book.head(3)

,book_title,rating_count,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00


In [62]:
# Now we are gonna set a condition for popular book
popular_books = popu_book[popu_book['rating_count']>=250].sort_values('avg_rating',ascending = False)
popular_books

,book_title,rating_count,avg_rating
80437,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80425,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80444,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80429,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80417,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
...,...,...,...
227714,Vinegar Hill (Oprah's Book Club (Paperback)),265,2.245283
233648,Whispers,286,2.199301
143383,Presumed Innocent,294,2.139456
94385,Isle of Dogs,288,2.000000


In [63]:
cols_include = ['book_title','isbn','book_author','publisher','Category',]
popular_books = popular_books.merge(prepro_data[cols_include], on='book_title').drop_duplicates('book_title')

In [65]:
n = int(input("Enter the number of top books that you want to see: "))
popular_books.head(n)

Enter the number of top books that you want to see:  10


,book_title,rating_count,avg_rating,isbn,book_author,publisher,Category
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,0439136350,J. K. Rowling,Scholastic,['Juvenile Fiction']
428,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,0439139597,J. K. Rowling,Scholastic,['Juvenile Fiction']
815,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,0590353403,J. K. Rowling,Scholastic,['Juvenile Fiction']
1093,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,043935806X,J. K. Rowling,Scholastic,['Juvenile Fiction']
1440,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,0439064864,J. K. Rowling,Scholastic,['Juvenile Fiction']
1996,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117,0345339681,J.R.R. TOLKIEN,Del Rey,9
2277,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370,0618002227,J. R. R. Tolkien,Houghton Mifflin Company,['Fiction']
2645,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652,059035342X,J. K. Rowling,Arthur A. Levine Books,['Juvenile Fiction']
3220,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769,0618002235,J. R. R. Tolkien,Houghton Mifflin Company,9
3480,To Kill a Mockingbird,510,4.700000,0446310786,Harper Lee,Little Brown & Company,['Fiction']


## Next we will work on Collaborative Filter Based Recommendation System, on this same dataset.